In [ ]:
%load_ext tensorboard

In [ ]:
import os
import glob
import numpy
import tensorflow as tf

from music21 import converter, instrument, note, chord
from keras.utils import np_utils

In [ ]:
notes = []

In [ ]:
for file in glob.glob("dataset/*.mid"):
    midi = converter.parse(file)

    notes_to_parse = None

    try:
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

with open('dataset/notes/parsed.data', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [ ]:
sequence_length = 100
n_vocab = len(set(notes))

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(
        512, 
        input_shape=(network_input.shape[1], network_input.shape[2]), 
        recurrent_dropout=0.3 
        return_sequences=True
    ),
    tf.keras.layers.LSTM(512, return_sequences=True, recurrent_dropout=0.3),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.BatchNorm(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Activation(tf.keras.activations.relu)
    tf.keras.layers.BatchNorm(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(n_vocab),
    tf.keras.layers.Activation(tf.keras.activations.softmax)
])

    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
%tensorboard --logdir logs --bind_all

In [ ]:
checkpoint_path = "models/weights-{epoch:02d}-{loss:.4f}.hdf5" 
log_path = os.path.join("logs", model_names[config_id - 1])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, 
    monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=[checkpoint_callback, tensorboard_callback])